## Telegram Automation:

* On Telegram, search @ BotFather, send him a “/start” message
* Send another “/newbot” message, then follow the instructions to setup a name and a username - Call it "NotifCall"
* Choose a username of my bot -> "NotifCall_bot"
* On Telegram, search your bot (by the username you just created), press the “Start” button or send a “/start” message
* Use this token to access the HTTP API:
                5121400289:AAHRZKx2voyY3HSfHJi2bgsLdHqnKB_WpcY
                Keep your token secure and store it safely, it can be used by anyone to control your bot.
* Open a new tab with your browser, enter https://api.telegram.org/bot<yourtoken>/getUpdates , replace <yourtoken> with                 your API token, press enter to run 
    https://api.telegram.org/bot5121400289:AAHRZKx2voyY3HSfHJi2bgsLdHqnKB_WpcY/getUpdates
* Done! Congratulations on your new bot. You will find it at t.me/NotifCall_bot. You can now add a description, about section and profile picture for your bot, see /help for a list of commands. By the way, when you've finished creating your cool bot, ping our Bot Support if you want a better username for it. Just make sure the bot is fully operational before you do this.


## Sending Stocker Status Data as File via Telegram  

In [ ]:
# File generation 
from datetime import datetime 
from os import path 
import pandas as pd 
import openpyxl
import requests
import sys

def telegram_bot_sendtext(bot_message, TELEGRAM_TOKEN, chat_id):
    bot_token = TELEGRAM_TOKEN
    bot_chatID = chat_id
    send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + bot_message

    response = requests.get(send_text)
    return response

def send_local_file(file_path, chat_id, TELEGRAM_TOKEN, API_URL):
    f = open(file_path, 'rb')
    file_bytes = f.read()
    f.close()
    response = {
        'document': (f.name, file_bytes),
        'chat_id': chat_id,
        'TELEGRAM_TOKEN' : TELEGRAM_TOKEN, 
        'API_URL': API_URL
    }
    method_name = 'sendDocument'
    return method_name, response

def send_response(method_name, params):
    API_URL = params['API_URL']
    if method_name == 'sendDocument':
        document = params['document']
        del params['document']
        r = requests.post(url=API_URL.format(method_name=method_name), params=params, files={'document': document})
    else:
        r = requests.post(url=API_URL.format(method_name=method_name), params=params)
    
    return r
#     return r.status_code == 200

def is_descending(list_):
    if len(list_) <= 1:
        return True
    # Strictement inférieur dans chaque enregistrement ou peut etre semblable dans des niveaux successives?
    if list_[0] < list_[1]:
        print({"LogInfo" : "the list values is not in a descending order !"})
        return False
    return is_descending(list_[1:])

def collect_values():
    counter = 0
    InputValuesDict = {}
    while counter <= 20:
        # verify inputs type
        try:
            NewInput = int(input("Enter the exact {} - th value of a the Stocker Status Please ! = > ".format(counter+1)))
        except ValueError:
            error_message = {"error": "XxX _ Log Error : the entred value is not valid !"}
            return error_message
        # assign the inputs to the dictionary with an entry date specification 
        InputValuesDict[str(datetime.now()).split('.')[0]] = NewInput
        counter += 1
    return InputValuesDict

def file_generation():
    InputDict = collect_values()
    FileInfo_Res  = {}
#     FileInfo_Res["path"] = "No path"
    FileInfo_Res["data"] = "No data"
    if "error" in InputDict:
        error_message = InputDict['error']
        return error_message
    
    values = list(InputDict.values())
    print('values : ', values)
    if is_descending(values):
        data = {'Date': list(InputDict.keys()), 'Values': list(InputDict.values())}
        data_frame = pd.DataFrame.from_dict(data)
        
        # Data Saving: Optional or not ? yes
#         Data_Saving_path = """.\\StockerStatus_Data.xlsx"""
#         FileInfo_Res["path"] = Data_Saving_path
#         data_frame.to_excel("""StockerStatus_Data.xlsx""")
        FileInfo_Res["data"] = data_frame
    
        return FileInfo_Res
    # The list is not in a descendinf order !
    else:
        return False
    
def sending_telegram_notification(file_path, chat_id, TELEGRAM_TOKEN, API_URL):
    file_generation_response  = file_generation()
    
    if file_generation_response == False:
        informational_message1 = {"LogInfo": "No warning detected !"}
        return informational_message1
    else:
        msg_entete = "Recap du : "+str(datetime.now()).split('.')[0]+"\n"
        bot_message = msg_entete+file_generation_response["data"].to_string()
        response = telegram_bot_sendtext(bot_message, TELEGRAM_TOKEN, chat_id)
        print({"LogInfo":"Sending process is Done ! - Response Status : {} ".format(response.status_code)})
        
# #   Methode to send the data frame stored on a .xlsx file 
#     if path.exists(file_generation_response["path"]):
# #        method_name, params = send_local_file(file_path, chat_id, TELEGRAM_TOKEN, API_URL)
#         response = send_response(method_name, params)
#         print({"LogInfo":"Sending process is Done ! - Response Status : {} ".format(response.status_code)})
#     else:
#         error_message = {"error": "saving data failure !"}
#         return error_message

In [ ]:
def main():
    file_path = ".\\StockerStatus_Data.xlsx"
    TELEGRAM_TOKEN = '5121400289:AAHRZKx2voyY3HSfHJi2bgsLdHqnKB_WpcY'
    API_URL = f'https://api.telegram.org/bot{TELEGRAM_TOKEN}' + '/{method_name}'
# # Only one receiver 
#     chat_id = '1104376640'
# # For a groupe of users grouped on a telegram groupe 
    chat_id = '-783940739'
    
    sending_telegram_notification(file_path, chat_id, TELEGRAM_TOKEN, API_URL)
    
if __name__ == '__main__':
    main()

## Simulation Caractère Stocker en temp réel :

* After creating the bot using instruction in the top of this page, then:
* Add the bot to the group: Go to the group, click on group name, click on Add members, 
    in the searchbox search for your bot like this: @my_bot, select your bot and click add. 
* open the group chat in a browser, you will see an URL in this format: "https://web.telegram.org/z/#-783940739" 
    a negative number occurs in the link, that is the chat_ID of the groupe that we can use to send our messages:
* we consider chat_id = -783940739, then we can use the telegram_bot_sendtext method to do the necessary to send notifs. 

In [ ]:
# File generation 
from datetime import datetime,timedelta
from os import path
from time import sleep
import pandas as pd
import numpy as np
import openpyxl
import requests
import sys

def telegram_bot_sendtext(bot_message, TELEGRAM_TOKEN, chat_id):
    bot_token = TELEGRAM_TOKEN
    bot_chatID = chat_id
    send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + bot_message

    response = requests.get(send_text)
    return response


def sending_telegram_notification(data_delivery, chat_id, TELEGRAM_TOKEN):
        msg_entete = " --------------- 🛑 Warning ⚠ --------------- \n\n 📑 Recap du : "+str(datetime.now()).split('.')[0]+"\n"
        # Message with full data description:
        bot_message_data = data_delivery.to_string()
        
        # Wanted & Significant Data:
        nbr_glass_on_stocker = list(data_delivery["Values"])[-1]
        time_to_stop_sec = 25 * nbr_glass_on_stocker
        splited_chreno = str(timedelta(seconds=time_to_stop_sec)).split(":")
        
        estimated_date_of_stop_date = datetime.now() + timedelta(seconds = int(time_to_stop_sec))
        estimated_date_of_stop_str  =  " 🚫 Next Stop: "+str(estimated_date_of_stop_date).split('.')[0]
        
        chreno_to_stop = """  {} :  houuurs |\ \n {} : minuutes | - > ⏳ To Stop  ! \n {} : seconds |/ """.format(splited_chreno[0], splited_chreno[1], splited_chreno[2])
        # Another way to avoid anotates eliminations
        # ar = np.array([[splited_chreno[0], " : ", "Heurs", "|\\", "", ""], [splited_chreno[1] ," : ", "Minutes ", " | ", " - - >", " To Stop 🛑 "], [splited_chreno[2], " : ", "Seconds", "|/", "", ""],])
        # df = pd.DataFrame(ar, index = [' ', ' ', ' '], columns = ['  ', '  ', '  ', '  ', ' ', ''])
        # chreno_to_stop = df.to_string()
        # print("chreno_to_stop : ", chreno_to_stop)

        bot_message = """{} \n {} \n\n {} """.format(msg_entete, chreno_to_stop, estimated_date_of_stop_str)        
        # print("bot_message : \n ", bot_message)

        # Send Warning Message : 
        response = telegram_bot_sendtext(bot_message, TELEGRAM_TOKEN, chat_id)
        
        print(response.text)
        print({"LogInfo":" Sending process ! - Response Status : {} \n\n".format(response.status_code)})
        
def is_descending(list_):
    if len(list_) <= 1:
        return True
    # Strictement inférieur dans chaque enregistrement ou peut etre semblable dans des niveaux successives? Res: oui 
    if list_[0] < list_[1]:
        print({"LogInfo" : "the list values is not in a descending order !"})
        return False
    return is_descending(list_[1:])
    
def preProcess_collected_data(chat_id, TELEGRAM_TOKEN):
    TimeIndicatorList = []
    ValuesList = []
    counter = 0
    InputValuesDict = {}
    
    while True:
        # verify inputs type & methode || ensure that the input value is in the correct format 
        testInput = 0
        while testInput == 0:
            try:
                NewInput = int(input("\n\n XxX _ Enter the exact {} - th value of a the Stocker Status Please ! = > ".format(counter+1)))
                testInput = 1
#                 print(" XxX _ {} entred value !  ".format(counter))
            except ValueError:
                error_message = {"error": "XxX _ Log Error : the entred value is not valid !"}
                print(error_message)

        # assign all inputs to a list with a date entry specification in another list 
        TimeIndicatorList.append(str(datetime.now()).split('.')[0])
        ValuesList.append(NewInput)
        
        if len(TimeIndicatorList) == len(ValuesList) and len(ValuesList) == 20:
            # Verify if values are descending or  not and continue the process 
            print({"LogInfo":"We get for the first time the desired level to continue the verification process in this level !"})
            
        elif len(TimeIndicatorList) == len(ValuesList) and len(ValuesList) > 20: 
            # Use the FIFO methode to garantie the data shape equals 20
            TimeIndicatorList = TimeIndicatorList[1:]
            ValuesList = ValuesList[1:]
            
        else:
            # Just continue appending values to the dataFrame   
            print('\n\n')
            counter += 1
            continue
        
        # continue the process after preparing data and limiting it's lenght on 20 elements 
        assert len(ValuesList) == 20, 'Only lists with a length of 20 are accepted for processing.'
        InputValuesDict = dict(zip(TimeIndicatorList, ValuesList))
        print("InputValuesDict -> ", InputValuesDict)
        values = list(InputValuesDict.values())
        print("Values  -> ", values)
        # verify the descendence of data to send a notification call !
        if is_descending(values):
            data = {'Date': list(InputValuesDict.keys()), 'Values': list(InputValuesDict.values())}
            data_delivery = pd.DataFrame.from_dict(data)
            print(data_delivery)
            print({"LogInfo":"Call the bot groupe, there is a work to do ! "})
            sending_telegram_notification(data_delivery, chat_id, TELEGRAM_TOKEN)
            # Sleep the process after sending the notification !
#             sleep(600)
        else:
            print({"LogInfo": "data input Nmbr : {} with the value : {}, no need for notification call, 👮: EverEverything's all right !".format(counter, NewInput)})
        counter += 1


In [ ]:
def main():
    TELEGRAM_TOKEN = '5121400289:AAHRZKx2voyY3HSfHJi2bgsLdHqnKB_WpcY'
# # For a groupe of users grouped on a telegram groupe 
    chat_id = '-783940739'
    # Origin !
    preProcess_collected_data(chat_id, TELEGRAM_TOKEN)
    
if __name__ == '__main__':
    main()

##### 

## 

In [ ]:
# # Test de la bibliothèque schedule
# import schedule
# import time

# def job():
#     print("I'm working...")

# schedule.every(10).seconds.do(job)
# schedule.every(10).minutes.do(job)
# schedule.every().hour.do(job)
# schedule.every().day.at("10:30").do(job)
# schedule.every(5).to(10).minutes.do(job)
# schedule.every().monday.do(job)
# schedule.every().wednesday.at("13:15").do(job)
# schedule.every().minute.at(":17").do(job)

# while True:
#     schedule.run_pending()
#     time.sleep(1)

In [ ]:
# # Draft
# import time
# import schedule
# import requests

# def telegram_bot_sendtext(bot_message):
#     bot_token = '5121400289:AAHRZKx2voyY3HSfHJi2bgsLdHqnKB_WpcY'
#     bot_chatID = '1104376640'
#     send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + bot_message

#     response = requests.get(send_text)
#     return response.json()

# def report():
#     my_balance = "hello word "   ## Replace this number with an API call to fetch your account balance
#     my_message = "Current balance is: {}".format(my_balance)   ## Customize your message
#     telegram_bot_sendtext(my_message)
#     print(my_message)

# # Faire un Running permanent configuré du script:
# # schedule.every().day.at("14:21").do(report)

# # while True:
# #     schedule.run_pending()
# #     time.sleep(1)

# # report()

# # test Also with : 
# TestDoc = '.\\Input_Data.xlsx'
# telegram_bot_sendtext(FileInfo_Res["data"].to_string())